In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification

url of folder of model=
url of dataset


model_path = './bert_classification_model'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)
model.eval()

def predict(post, comment):
    inputs = tokenizer.encode_plus(
        post,
        comment,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
    return predicted_class

input_csv = './39000unlabeled_data2.csv'  #input csv

data = pd.read_csv(input_csv)

label_mapping = {0: -1, 1: 0, 2: 1}

for index, row in data.iterrows():
    post = row['cleaned_text']
    comment = row['cleaned_body']
    predicted_class = predict(post, comment)
    data.at[index, 'label_agreement'] = label_mapping[predicted_class]

data.to_csv(input_csv, index=False)
print(f"Predictions updated in {input_csv}")


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
# First, mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install gdown if not already installed
!pip install gdown

import gdown
import os
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from tqdm import tqdm




import torch
print(torch.cuda.is_available())  # Should return True



# Create directories for downloaded files
!mkdir -p /content/model
!mkdir -p /content/data

# Download the dataset (tendimension2_zeroshot)
dataset_url = "https://drive.google.com/uc?id=16AwDboS2sEq2zR2nZtDYNmBqm6WkAKYU"
dataset_path = '/content/data/tendimension2_zeroshot.csv'
gdown.download(dataset_url, dataset_path, quiet=False)

# Download the model folder (bert_classification_model)
model_folder_id = "1HBWtmkIb9W7q2Fk8yCefK3t2QZNjrm7R"
model_path = '/content/model/bert_classification_model'
!gdown --folder https://drive.google.com/drive/folders/{model_folder_id} -O {model_path}

# Set output path
output_path = '/content/drive/MyDrive/labeled_tendimension2_zeroshot.csv'  # Added descriptive name for output

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load model and move to GPU
try:
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertForSequenceClassification.from_pretrained(model_path)
    model = model.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    model.eval()
    print("Model loaded successfully")
except Exception as e:
    print(f"Error loading model: {e}")
    print(f"Contents of model directory: {os.listdir(model_path)}")

def predict_batch(posts, comments, batch_size=32):
    predictions = []

    for i in tqdm(range(0, len(posts), batch_size), desc="Processing batches"):
        batch_posts = posts[i:i + batch_size]
        batch_comments = comments[i:i + batch_size]

        inputs = tokenizer.batch_encode_plus(
            list(zip(batch_posts, batch_comments)),
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        # Move inputs to GPU
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            batch_predictions = torch.argmax(logits, dim=1).cpu().numpy()
            predictions.extend(batch_predictions)

    return predictions

# Read the dataset
print("Loading dataset...")
try:
    data = pd.read_csv(dataset_path)
    print(f"Dataset loaded successfully with {len(data)} rows")
    print("Columns in dataset:", data.columns.tolist())
except Exception as e:
    print(f"Error loading dataset: {e}")

# Prepare data for batch processing
posts = data['cleaned_text'].tolist()
comments = data['cleaned_body'].tolist()

# Define label mapping
label_mapping = {0: -1, 1: 0, 2: 1}

# Process in batches with progress bar
print("Processing predictions...")
predictions = predict_batch(posts, comments)

# Map predictions to labels
data['agree_label'] = [label_mapping[pred] for pred in predictions]

# Save the results
print("Saving results...")
data.to_csv(output_path, index=False)
print(f"Predictions saved to {output_path}")

# Print statistics
print("\nPrediction Statistics:")
print(data['agree_label'].value_counts())

Mounted at /content/drive
True


Downloading...
From (original): https://drive.google.com/uc?id=16AwDboS2sEq2zR2nZtDYNmBqm6WkAKYU
From (redirected): https://drive.google.com/uc?id=16AwDboS2sEq2zR2nZtDYNmBqm6WkAKYU&confirm=t&uuid=d4a4fe70-e6ad-458b-9276-5c3173845fe0
To: /content/data/tendimension2_zeroshot.csv
100%|██████████| 501M/501M [00:04<00:00, 124MB/s]


Retrieving folder contents
Processing file 1mJhi64S94Dv-xBCPKUSUPCFPSoDdPfrY config.json
Processing file 1NSDem6rQ-SCR6TUAPH9SeAf-zGP26kQu pytorch_model.bin
Processing file 1qenA5WwzkVhQ21vbX5GpwxhwErhTYPPS special_tokens_map.json
Processing file 1hHEiFSzRBy1D18i3Rrl_di056dhtNWJd tokenizer_config.json
Processing file 1hCkTkApQf0a3E2wacErdeZkAdYwRJtNh vocab.txt
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1mJhi64S94Dv-xBCPKUSUPCFPSoDdPfrY
To: /content/model/bert_classification_model/config.json
100% 881/881 [00:00<00:00, 5.13MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1NSDem6rQ-SCR6TUAPH9SeAf-zGP26kQu
From (redirected): https://drive.google.com/uc?id=1NSDem6rQ-SCR6TUAPH9SeAf-zGP26kQu&confirm=t&uuid=628d9861-e3c6-49db-b8b0-a9b5da513d3a
To: /content/model/bert_classification_model/pytorch_model.bin
100% 438M/438M [00:03<00:00, 134MB/s]
Downloading...
Fro

Streaming output truncated to the last 5000 lines.
Processing batches:  77%|███████▋  | 1090/1418 [29:33<08:20,  1.53s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some 

Saving results...
Predictions saved to /content/drive/MyDrive/labeled_tendimension2_zeroshot.csv

Prediction Statistics:
agree_label
-1    28708
 0     9877
 1     6782
Name: count, dtype: int64
